# Assignment 2

Lucas Bezerra, ID: 171412,
lucas.camaradantasbezerra@kaust.edu.sa

## Problem 1: Transformer Questions

<strong>Q1 (10 points): In the above self-attention operation, why we need to incorporate the scale factor √dk into calculation?</strong>

The scale factor is used for normalizing the dot product $Q\,K^T$, making sure it has the same variance as the individual components of $Q$ and $K$.

Assume these components are random variables $q_i, k_i\;,\forall i\in\{1,2,\dots,d_k\}$ with mean 0 and variance $\sigma^2$. Then, it stems from the Central Limit Theorem that the dot product between $\mathbf{q}$ and $\mathbf{k}$ is normally-distributed:

$$ \mathbf{q}\cdot \mathbf{k} = \sum\limits_{i=1}^{d_k} q_i\cdot k_i \sim \mathcal{N}\left( 0, d_k\cdot \sigma^2 \right) $$

$$ \frac{1}{\sqrt{d_k}} \mathbf{q}\cdot \mathbf{k} \sim \mathcal{N}\left( 0,\sigma^2 \right) $$

This ensures the network will have similar ranges of parameters no matter what dimensionality is chosen, and avoids vanishing gradient problems (keeping the variance of $\mathbf{q}\cdot \mathbf{k}$ low ensures that $\text{softmax}(\mathbf{q}\cdot \mathbf{k})$ is also close to the origin, and thus in a high-gradient region).

<strong>Q2 (10 points): When we train the Transformer on the word sequences, usually we need to add additional positional embedding for each word, why is this necessary?</strong>

Without position encoding, the attention is an ordering-agnostic function, meaning that the ordering of words in a sentence does not change the outcome. However, in any language the order of words matters. That is why a positional embedding is added to each word embedding.

<strong>Q3 (10 points): In the Transformer framework, there are two types of attention modules, which are self-attention and encoder-decoder attention. What is the difference between these two modules in terms of functionality and technical implementation?</strong>

TO DO ---------------------------------

<strong>Q4 (10 points): There are also other types of attention calculations such as the additive attention [4]. Additive attention computes the compatibility function using a feed-forward network with a single hidden layer. In the Transformer model, why the authors choose to use scaled-dot product attention instead of additive attention and what is the main advantages?</strong>

In [2] the authors argue that, although both attention mechanisms are similar in theoretical complexity, the multiplicative attention is computationally more efficient, since it consists simple matrix multiplications, an operation that runs very fast in specialized hardware (e.g. GPUs).

<strong>Q5 (10 points): BERT and GPT model pretrain their model on a largescale dataset in a self-supervising way. Please describe their pretraining tasks and discuss why it is useful.</strong>

TO DO ---------------------------------

<strong>Q6 (10 points) : In the BERT model design, there are two special tokens \[CLS\] and [SEP], what is the purpose of designing these two special tokens and how they are used during the training and evaluation.</strong>

TO DO ---------------------------------

## Problem 2: Coding Implementation

In [1]:
import os,sys,time,math,textwrap

import numpy as np

import torch
import torch.nn as nn

import dataset, transformer

root = 'data/wikitext-2'

In [ ]:
lr = .00035
context = 150
batch_size = 32
log_interval = 50

heads = 10
depth = 16

torch.manual_seed(0)
device = torch.device("cuda")

In [ ]:
train_data = dataset.WikiText2(root, context, dataset.DatasetSplit.train)
valid_data = dataset.WikiText2(root, context, dataset.DatasetSplit.valid)
test_data = dataset.WikiText2(root, context, dataset.DatasetSplit.test)

In [ ]:
def evaluate(data):
    model.eval()
    with torch.no_grad():
        loss = 0.
        loader = torch.utils.data.DataLoader(dataset=data,batch_size=batch_size,shuffle=False)
        for i, (x,y) in enumerate(loader):
            x, y = x.permute(1,0).to(device), y.permute(1,0).to(device)
            yhat = model(x).view(-1, train_data.word_count())
            loss += criterion(yhat, y.contiguous().view(-1))

    print()
    model.train()
    return loss / len(loader)

In [ ]:
model = transformer.Transformer(context, train_data.word_count(), 400, 40, 900, heads, depth, tied_weights=True).to(device)
count = sum([np.prod(parm.shape) for parm in model.parameters() if parm.requires_grad])
print('Initialized graph with {} parameters'.format(count))

In [ ]:
criterion = nn.NLLLoss()
curr_lr = .0001
clip = .25
best_val_loss = None
epochs = 10
save = 'model.pt'

train_loader = torch.utils.data.DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
print('Initiating training, {} iterations/epoch.'.format(len(train_loader)))

try:
    optimizer = torch.optim.Adam(model.parameters(), lr=curr_lr)
    for epoch in range(epochs):
        t0 = time.time()
        val_loss = evaluate(valid_data)
        print('-' * 100)
        print('| checkpoint | epoch {:3d} | time: {:5.2f}s | validation loss {:5.2f} | '
                'validation perplexity {:8.2f}'.format(epoch, (time.time() - t0),
                                                       val_loss, math.exp(val_loss)))
        print('-' * 100)
        print('epoch\t\tms/batch\tlr\tloss\tperplexity')

        if not best_val_loss or val_loss < best_val_loss:
            with open(save, 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss

        model.train()
        total_loss = 0.
        t0 = time.time()
        if epoch == 1: optimizer.param_groups[0]['lr'] = curr_lr = lr # finished warmup
        for i, (x,y) in enumerate(train_loader):
            if i % log_interval == 0 and i > 0:
                cur_loss = total_loss / log_interval
                elapsed = time.time() - t0
                print('{:3d} ({:2.1f}%)\t{:5.2f}\t\t{:1.3}\t{:5.2f}\t{:8.2f}'.format(
                    epoch, 100*i/float(len(train_loader)),
                    elapsed * 1000 / log_interval, curr_lr, cur_loss, math.exp(cur_loss)))
                total_loss = 0
                t0 = time.time()

            x, y = x.permute(1,0).to(device), y.permute(1,0).to(device)
            model.zero_grad()
            yhat = model(x).view(-1, train_data.word_count())
            loss = criterion(yhat, y.contiguous().view(-1))
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()

            total_loss += loss.item()

except KeyboardInterrupt:
    print('Graceful Exit')

In [ ]:
print('Restoring best checkpointed model...')
with open(save, 'rb') as f:
    model = torch.load(f)

test_loss = evaluate(test_data)
print('=' * 89)
print('| end of training | test loss {:5.2f} | test perplexity {:8.2f}'.format(test_loss, math.exp(test_loss)))
print('=' * 89)

In [ ]:
print('\nUncurated samples')
print('-' * 89)

def sample():
    words = []
    model.eval()
    history = torch.randint(train_data.word_count(), (1, 1), dtype=torch.long).cuda()
    for i in range(context):
        output = model(history)
        word_weights = output[-1].squeeze().exp().cpu()
        word_idx = torch.multinomial(word_weights, 1)[0]
        word_tensor = torch.Tensor([[word_idx]]).long().cuda()
        history = torch.cat([history, word_tensor], 0)

        words.append(train_data.idx2word[word_idx])

    return '\n'.join(textwrap.wrap(' '.join(words),80))

for i in range(5):
    print('({})'.format(i), sample())